#### Attention:
Ceci ne prend pas en compte les départements d'outre mer mais uniquement la métrople !! (je n'ai pas trouvé de carte fonctionnel)

In [2]:
%pip install altair pandas numpy plotly geopandas requests
import altair as alt 
alt.data_transformers.enable('json')

Note: you may need to restart the kernel to use updated packages.


DataTransformerRegistry.enable('json')

In [3]:
import pandas as pd
import plotly.graph_objs as go
import numpy as np

In [4]:
df = pd.read_csv("data/dpt2020.csv", sep=";")

# Vérifier si '2A' ou '2B' sont présents
print("'2A' présent ?", '2A' in df['dpt'].unique())
print("'2B' présent ?", '2B' in df['dpt'].unique())

'2A' présent ? False
'2B' présent ? False


In [5]:
print(f"Initial shape is {df.shape}")

# supp years and dpt = XXXX and transform to int
to_clean=['annais', 'dpt']
for col in to_clean:
    #df[col] = df[col].astype(str).str.strip()
    
    # Supprimer les lignes où la colonne n'est pas strictement numérique
    df = df[df[col].str.isnumeric()]
    
    if col == 'annais':
        df[col] = df[col].astype(int)
        
    else:
    # Reforcer le type string natif (facultatif mais propre)
        df[col] = df[col].astype("string")
        
        ### Ce n'est pas supported par la carte que nous avons ###
        # Transformation pour la corse => tout est transformé en 2A
        df.loc[df['dpt'] == '20', 'dpt'] = '2A'
        # Exclusion des DOM_TOM
        df = df[~df['dpt'].isin(['971', '972', '973', '974'])]
    print(df[col].dtype)
    print(f"Shape is now {df.shape}")

Initial shape is (3727553, 5)
int64
Shape is now (3690309, 5)
string
Shape is now (3492154, 5)


In [6]:
# Supprimer les prénoms "_PRENOMS_RARES"
df = df[df["preusuel"] != "_PRENOMS_RARES"]
# Ajouter une colonne de longueur de prénom
df["longueur"] = df["preusuel"].str.len()
# Supprimer les prénoms de longueur 1
df = df[df["longueur"] > 1]
print(f"Shape is now {df.shape}")

Shape is now (3471072, 6)


In [7]:
df

,sexe,preusuel,annais,dpt,nombre,longueur
10885,1,AADIL,1983,84,3,5
10886,1,AADIL,1992,92,3,5
10888,1,AAHIL,2016,95,3,5
10892,1,AARON,1962,75,3,5
10893,1,AARON,1976,75,3,5
...,...,...,...,...,...,...
3727543,2,ZYA,2011,91,3,3
3727545,2,ZYA,2013,44,4,3
3727546,2,ZYA,2013,59,3,3
3727548,2,ZYA,2018,59,3,3


In [8]:
max_nombre = df.groupby(['sexe', 'annais', 'dpt'])['nombre'].transform('max')
df_final = df[df['nombre'] == max_nombre]
# Tri décroissant puis suppression des duplicatas
df_final = (
    df.sort_values('nombre', ascending=False)
      .drop_duplicates(['sexe', 'annais', 'dpt'])
      .sort_values(by=['sexe', 'annais', 'dpt'])
)
moyenne_longueur = df.groupby(['sexe', 'annais', 'dpt'])['longueur'].transform('mean').round().astype(int)
df_final['moyenne_longueur'] = moyenne_longueur

In [9]:
df_final

,sexe,preusuel,annais,dpt,nombre,longueur,moyenne_longueur
770685,1,JEAN,1900,01,144,4,6
114883,1,ANDRÉ,1900,02,138,5,6
770687,1,JEAN,1900,03,230,4,6
1023835,1,LOUIS,1900,04,40,5,6
900377,1,JOSEPH,1900,05,50,6,6
...,...,...,...,...,...,...,...
2864122,2,LINA,2020,91,69,4,5
2377520,2,EMMA,2020,92,132,4,6
2864124,2,LINA,2020,93,101,4,6
2864125,2,LINA,2020,94,87,4,6


In [10]:
df_final[(df_final['annais'] == 2018) & (df_final['dpt'] == '46')]

,sexe,preusuel,annais,dpt,nombre,longueur,moyenne_longueur
590549,1,GABRIEL,2018,46,10,7,5
2816747,2,LÉA,2018,46,6,3,4


In [11]:
import geopandas as gpd
from io import BytesIO
import requests

#Préparation de la carte
url = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
response = requests.get(url)
response.raise_for_status()
gdf = gpd.read_file(BytesIO(response.content))
gdf["geometry"] = gdf["geometry"].simplify(0.01)

dep = gdf.merge(df_final, how='right', left_on='code', right_on='dpt')
dep['moyenne_longueur'] = dep['moyenne_longueur'].fillna(0).astype(int)
dep = dep.dropna(subset=['geometry'])

In [14]:
#Widget pour la carte
# Paramètre pour l'année (avec un slider)
year_param = alt.selection_point(
    fields=["annais"],
    bind=alt.binding_range(
        min=dep['annais'].min(),
        max=dep['annais'].max(),
        step=1,
    name="Année :"
    ),
    value=dep['annais'].min()
)

# Paramètre pour le sexe (avec bouton radio)
gender_param = alt.selection_point(
    fields=["sexe"],
    bind=alt.binding_radio(
        name="Sexe :", 
        options=[1, 2],
        labels=['♂ Homme', '♀ Femme']
    ),
    value=1
)

# Paramètre pour la longueur (avec liste déroulante)
length_param = alt.selection_point(
    fields=["moyenne_longueur"], 
    bind=alt.binding_select(
        options=sorted(dep["moyenne_longueur"].dropna().unique()), 
        name="Longueur moyenne : "
    ),
    value=dep["moyenne_longueur"].iloc[0]  # Valeur par défaut
)

carte = alt.Chart(dep).mark_geoshape(
    stroke='white'
).encode(
    color=alt.Color("moyenne_longueur:Q", scale=alt.Scale(scheme='viridis'), title="Longueur moyenne"),
    tooltip=[
        alt.Tooltip("nom:N", title="Département"),
        alt.Tooltip("code:N", title="Code"),
        alt.Tooltip("moyenne_longueur:Q", title="Longueur moyenne du prénom"),
        alt.Tooltip("preusuel:N", title="Prénom le plus populaire")]
).add_params(
    gender_param,
    year_param
).transform_filter(
    gender_param,
    year_param
).project(
    type='mercator'
).properties(
    width=600,
    height=600
)

carte


alt.Chart(...)

In [13]:
dep[(dep['annais'] == 2018) & (dep['dpt'] == '46')]

,code,nom,geometry,sexe,preusuel,annais,dpt,nombre,longueur,moyenne_longueur
10915,46,Lot,"POLYGON ((1.40926 45.006, 1.53572 45.04628, 1....",1,GABRIEL,2018,46,10,7,5
22070,46,Lot,"POLYGON ((1.40926 45.006, 1.53572 45.04628, 1....",2,LÉA,2018,46,6,3,4
